## Create User Table Size
This table contains data about
- table size in bytes and giga bytes
- row count
- when table was created and updated
- what is the number of files per table
- what is estimated size per file 
- partitioned columns
- clutered columns
- statistics

Preparing the fundation, widgets to setup parameteres and imports

In [ ]:
dbutils.widgets.text('target_schema', 'next_level_dm')
dbutils.widgets.text('user_tab_size_table_name', 'dbx_user_tab_size')

In [ ]:
in_target_schema = dbutils.widgets.get('target_schema')
in_user_tab_size = dbutils.widgets.get('user_tab_size_table_name')

In [ ]:
from pyspark.sql.functions import col, lit

getting list of a table from schema, it requires dbx_user_tab_columns table to be creted, check user_tab_columns.ipynb

Filtering out views, user and temp tables

In [ ]:
all_tables = spark.sql(f"select distinct table_name, substr(table_name,1,locate('_', table_name)-1) as table_prefix from {in_target_schema}.dbx_user_tab_columns where table_name not like 'v%' and table_name not like 'dbx%' and table_name not like 'z%' ").collect()

runnig describe detail on tables listed above

In [ ]:
table_size_df = None
for table in all_tables:
    table_name = table["table_name"]
    if spark.catalog.tableExists(f"{in_target_schema}.{table_name}"):
        table_details = spark.sql(f"describe detail {in_target_schema}.{table_name}")
        nbr_of_rows = spark.sql(f"select count(*) as cnt from {in_target_schema}.{table_name}").collect()[0]["cnt"]
        table_details = table_details.withColumn("size_in_gb",col("sizeInBytes")/(1024*1024*1024)).withColumn('size_per_file', col("sizeInBytes")/col("numFiles")).withColumn("nbr_of_rows",lit(nbr_of_rows)).withColumn("table_name",lit(table_name)).withColumn("table_prefix",lit(table["table_prefix"]))
        table_details = table_details.selectExpr("table_name","table_prefix","sizeInBytes as size_in_bytes","size_in_gb", 
                                                 "numFiles as nbr_of_files","createdAt as created_at","lastModified as last_modified_at", "statistics","partitionColumns as partition_columns","clusteringColumns as clustering_columns","nbr_of_rows","size_per_file")

        table_size_df = table_details if table_size_df is None else table_size_df.unionAll(table_details)



In [ ]:
table_size_df.createOrReplaceTempView('v_user_tab_size')

In [ ]:
spark.sql(f"""create or replace table {in_target_schema}.{in_user_tab_size} as 
            select us.*, current_timestamp() as load_date from v_user_tab_size us
            """)

In [ ]:
display(table_size_df)